# Single Neuron Parameter Study
> Take the 1 neuron case, and vary its parameters (do a bifurcation analysis, attack computationally, ignore the neuralness).  Main goal is to determine where/whether bi-stable vs single impulse steady states exist